# Project Disagreement

### Step 1: Search for documents in the [DIMENSIONS](https://app.dimensions.ai/discover/publication) database and create an Excel-file containing search results ("search_results.xlsx").
* Run a query in the search bar. 

(I chose search option "DOI" and searched for known DOIs. One of my documents has no DOI, so I chose search option "Title and abstract" and searched for some sentences of this document's "SUMMARY" section.)

* Klick on "Save/Export" (on the right next to the search bar) and export the list of search results. Choose to export the results as an Excel-file.

(In this way I created "hcq.xlsx".)

### Step 2: Make a Dataframe out of the Excel-file

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [2]:
df_hcq = pd.read_excel("hcq.xlsx",skiprows=[0])

In [3]:
df_hcq

,Rank,Publication ID,DOI,PMID,PMCID,Title,Source title,Anthology title,MeSH terms,Publication Date,...,Corresponding Author,Authors Affiliations,Times cited,Recent citations,RCR,FCR,Source Linkout,Dimensions URL,FOR (ANZSRC) Categories,Sustainable Development Goals
0,113,pub.1126880632,10.1186/s12969-020-00422-z,32321540.0,PMC7175817,COVID-19 and what pediatric rheumatologists sh...,Pediatric Rheumatology,NaN,"Adolescent; Anti-Inflammatory Agents, Non-Ster...",2020-12,...,"Giani, Teresa (Meyer Children's Hospital; Univ...","Licciardi, Francesco (University of Turin); Gi...",1,1,NaN,NaN,https://ped-rheum.biomedcentral.com/track/pdf/...,https://app.dimensions.ai/details/publication/...,1108 Medical Microbiology; 11 Medical and Heal...,NaN
1,65,pub.1127125006,10.1186/s13054-020-02894-7,32345336.0,PMC7187670,Shining a light on the evidence for hydroxychl...,Critical Care,NaN,Betacoronavirus; Coronavirus Infections; Human...,2020-12,...,"Ingraham, Nicholas E. (University of Minnesota)","Ingraham, Nicholas E. (University of Minnesota...",0,0,NaN,NaN,https://ccforum.biomedcentral.com/track/pdf/10...,https://app.dimensions.ai/details/publication/...,11 Medical and Health Sciences,NaN
2,58,pub.1125978056,10.1016/j.medmal.2020.03.006,32240719.0,PMC7195369,No Evidence of Rapid Antiviral Clearance or Cl...,Médecine et Maladies Infectieuses,NaN,Adult; Aged; Antiviral Agents; Azithromycin; B...,2020-06,...,"Molina, Jean Michel (Hôpital Saint-Louis)","Molina, Jean Michel (Hôpital Saint-Louis); Del...",67,67,NaN,NaN,https://doi.org/10.1016/j.medmal.2020.03.006,https://app.dimensions.ai/details/publication/...,NaN,NaN
3,109,pub.1127834352,10.1016/s0140-6736(20)31180-6,32450107.0,PMC7255293,Hydroxychloroquine or chloroquine with or with...,The Lancet,NaN,NaN,2020-05,...,"Mehra, Mandeep R (Brigham and Women's Hospital)","Mehra, Mandeep R (Brigham and Women's Hospital...",7,7,NaN,NaN,https://doi.org/10.1016/s0140-6736(20)31180-6,https://app.dimensions.ai/details/publication/...,1117 Public Health and Health Services; 11 Med...,3 Good Health and Well Being
4,20,pub.1126667578,10.1101/2020.04.10.20060558,NaN,NaN,Hydroxychloroquine in patients mainly with mil...,medRxiv,NaN,NaN,2020-05,...,NaN,"Tang, Wei (Shanghai Jiao Tong University); Cao...",33,33,NaN,NaN,https://www.medrxiv.org/content/medrxiv/early/...,https://app.dimensions.ai/details/publication/...,1117 Public Health and Health Services; 11 Med...,NaN
5,0,pub.1125404383,10.1016/j.antiviral.2020.104762,32147496.0,PMC7132364,Of chloroquine and COVID-19,Antiviral Research,NaN,Antiviral Agents; Betacoronavirus; China; Chlo...,2020-05,...,"de Lamballerie, Xavier (Méditerranée Infection...","Touret, Franck (Méditerranée Infection Foundat...",67,67,NaN,NaN,https://doi.org/10.1016/j.antiviral.2020.104762,https://app.dimensions.ai/details/publication/...,1108 Medical Microbiology; 11 Medical and Heal...,NaN
6,63,pub.1127182972,10.4414/smw.2020.20262,32347963.0,NaN,An independent appraisal and re-analysis of hy...,Swiss Medical Weekly,NaN,Adult; Azithromycin; Betacoronavirus; Clinical...,2020-04-29,...,NaN,"Intson, Katheron (University of Toronto); Kuma...",0,0,NaN,NaN,https://smw.ch/journalfile/view/article/ezm_sm...,https://app.dimensions.ai/details/publication/...,11 Medical and Health Sciences; 1103 Clinical ...,NaN
7,19,pub.1126839717,10.1101/2020.04.14.20065276,NaN,NaN,Hydroxychloroquine Versus COVID-19: A Periodic...,medRxiv,NaN,NaN,2020-04-20,...,NaN,"Shamshirian, Amir (Mazandaran University of Me...",4,4,NaN,NaN,https://www.medrxiv.org/content/medrxiv/early/...,https://app.dimensions.ai/details/publication/...,1117 Public Health and Health Services; 11 Med...,3 Good Health and Well Being
8,21,pub.1126655433,10.1101/2020.04.10.20060699,NaN,NaN,No evidence of clinical efficacy of hydroxychl...,medRxiv,NaN,NaN,2020-04-14,...,NaN,"Mahevas, Matthieu (); Tran, Viet-Thi (); Roumi...",27,27,NaN,NaN,https://www.medrxiv.org/content/medrxiv/early/...,https://app.dimensions.ai/details/publication/...,111

## Exploit Websites

In [4]:
pubkeys = [
    'title',
    'aff_org_name',
    'researcher_dim_id',
    'researcher_dim_count',
    'journal_title',
    'language',
    'abstract',
    'open_access',
    'publisher',
    'aff_city_name',
    'aff_country_name',
    'doi',
    'pub_date',
    'pub_year',
    'times_cited',
    'altmetric_id',
    'altmetric',
    'authors_full'
    ]

In [5]:
def getmeta(x):
    html = ""
    d = ""
    value = ""
    url = x["Dimensions URL"]
    
    if url != None:
        html = requests.get(url).content
        
    soup = BeautifulSoup(html, "lxml")
    datadoc = soup.find("div")
    
    if datadoc == None:
        attr = ""
    else:
        attr = datadoc.get("data-doc")
        if attr:
            d = json.loads(attr)
    
    dicpub = dict()
    
    for i in pubkeys:
        if d != "":
            value = d.get(i, "NaN")
        dicpub[i] = value
        
    return(dicpub)

In [6]:
df_hcq["seldict"] = df_hcq.apply(getmeta, axis=1)

In [7]:
lidi = []
def newpub(x):
    di = dict()
    d = x["seldict"]
    di["DOI"] = x["DOI"]
    di["title"] = x["Title"]
    di["authors"] = d["authors_full"]
    di["publisher"] = d["publisher"]
    di["source"] = x["Source title"]
    di["aff_org_name"] = d["aff_org_name"]
    di["aff_country"] = d["aff_country_name"]
    di["pub_date"] = d["pub_date"]
    di["abstract"] = d["abstract"]
    di["openaccess"] = d["open_access"]
    di["di_URL"] = x["Dimensions URL"]
    lidi.append(di)

In [8]:
df_hcq.apply(lambda x:newpub(x), axis=1)
df_hcq_Lit = pd.DataFrame(lidi)
df_hcq_Lit.head()

,DOI,title,authors,publisher,source,aff_org_name,aff_country,pub_date,abstract,openaccess,di_URL
0,10.1186/s12969-020-00422-z,COVID-19 and what pediatric rheumatologists sh...,"[Francesco Licciardi, Teresa Giani, Letizia Ba...",Springer Nature,Pediatric Rheumatology,"[University of Siena, University of Turin, Div...",[Italy],2020-12,"On March 11th, 2020 the World Health Organizat...",True,https://app.dimensions.ai/details/publication/...
1,10.1186/s13054-020-02894-7,Shining a light on the evidence for hydroxychl...,"[Nicholas E. Ingraham, David Boulware, Matthew...",Springer Nature,Critical Care,"[University of North Carolina at Chapel Hill, ...",[United States],2020-12,,True,https://app.dimensions.ai/details/publication/...
2,10.1016/j.medmal.2020.03.006,No Evidence of Rapid Antiviral Clearance or Cl...,"[Jean Michel Molina, Constance Delaugerre, Jer...",Elsevier,Médecine et Maladies Infectieuses,"[University of Paris-Sud, Hôpital Saint-Louis,...",[France],2020-06,,True,https://app.dimensions.ai/details/publication/...
3,10.1016/s0140-6736(20)31180-6,Hydroxychloroquine or chloroquine with or with...,"[Mandeep R Mehra, Sapan S Desai, Frank Ruschit...",Elsevier,The Lancet,"[Brigham and Women's Hospital, University Hosp...","[Switzerland, United States]",2020-05,"BACKGROUND: Hydroxychloroquine or chloroquine,...",True,https://app.dimensions.ai/details/publication/...
4,10.1101/2020.04.10.20060558,Hydroxychloroquine in patients mainly with mil...,"[Wei Tang, Zhujun Cao, Mingfeng Han, Zhengyan ...",NaN,medRxiv,"[Department of Infectious Disease, The Sixth P...",[China],2020-05,Abstract Objectives To assess the efficacy and...,True,https://app.dimensions.ai/details/publication/...


In [9]:
hcq_abstracts = df_hcq_Lit["abstract"]
hcq_abstracts

0     On March 11th, 2020 the World Health Organizat...
1                                                      
2                                                      
3     BACKGROUND: Hydroxychloroquine or chloroquine,...
4     Abstract Objectives To assess the efficacy and...
5     Recent publications have brought attention to ...
6     A recent open-label study claimed that hydroxy...
7     Abstract Importance: This systematic review an...
8     Background Treatments are urgently needed to p...
9      Background There is no specific antiviral the...
10    The ongoing pandemic coronavirus disease 19 (C...
11    Use of these drugs is premature and potentiall...
12    OBJECTIVE: The novel severe acute respiratory ...
13    Chloroquine — an approved malaria drug — is kn...
14    Aims: Studies have indicated that chloroquine ...
15    BACKGROUND: Chloroquine and hydroxychloroquine...
16    The coronavirus disease 2019 (COVID-19) virus ...
17    The current pandemic coronavirus disease 2

In [10]:
for i in range(len(hcq_abstracts)):
    print(hcq_abstracts.iloc[i])

On March 11th, 2020 the World Health Organization declared COVID-19 a global pandemic. The infection, transmitted by 2019 novel coronavirus (2019-nCov), was first discovered in December 2019, in Wuhan, Hubei Province, and then rapidly spread worldwide. Italy was early and severely involved, with a critical spread of the infection and a very high number of victims. Person-to-person spread mainly occurs via respiratory droplets and contact. The median incubation period is 5 days. The spectrum of respiratory symptoms may range from mild to severe, strictly depending on the age of the patient and the underlying comorbidities.In children COVID-19 related disease is less frequent and less aggressive. In Italy 1% of positive cases are under 18 years of age, and no deaths have been recorded before 29 years of age. For patients affected by rheumatic disease, despite the concerns related to the imbalance of their immune response and the effect of immunosuppressive treatments, there are still few